### **Análise dos dados**

Segue abaixo a análise dos dados inicialmente fornecidos.

Imports

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.metrics import confusion_matrix
import seaborn as sns

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# LSTM Teste 1 

In [5]:
df = pd.read_csv('../data/preparacao_modelo_1.csv')
df

,KNR,ID1NAME,ID1SOK,ID1SNOK,ID1DATA,ID2NAME,ID2SOK,ID2SNOK,ID2DATA,ID718NAME,ID718SOK,ID718SNOK,ID718DATA,FALHA
0,2023-2016173,0.0,0.0,0.0,0.0,0.010733,0.010791,0.0,3.736757e-06,0.005038,0.003546,0.004405,0.000047,0.0
1,2023-2026098,0.0,0.0,0.0,0.0,0.007156,0.007194,0.0,1.868379e-07,0.002519,0.003546,0.000000,0.000000,0.0
2,2023-2026162,0.0,0.0,0.0,0.0,0.007156,0.007194,0.0,9.341893e-07,0.002519,0.003546,0.000000,0.000000,0.0
3,2023-2026175,0.0,0.0,0.0,0.0,0.007156,0.007194,0.0,1.121027e-06,0.002519,0.003546,0.000000,0.000000,0.0
4,2023-2026215,0.0,0.0,0.0,0.0,0.007156,0.007194,0.0,9.341893e-07,0.002519,0.003546,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49186,2024-2976009,0.0,0.0,0.0,0.0,0.064401,0.064748,0.0,7.374491e-04,0.000000,0.000000,0.000000,0.000000,0.0
49187,2024-2976010,0.0,0.0,0.0,0.0,0.064401,0.064748,0.0,9.554889e-04,0.000000,0.000000,0.000000,0.000000,1.0
49188,2024-2976011,0.0,0.0,0.0,0.0,0.066190,0.066547,0.0,1.438091e-03,0.000000,0.000000,0.000000,0.000000,1.0
49189,2024-2976012,0.0,0.0,0.0,0.0,0.073345,0.073741,0.0,6.931648e-02,0.000000,0.000000,0.000000,0.000000,1.0


In [6]:
# Separando as features (X) e o target (y)
X = df.drop(columns=['FALHA', 'KNR'])  # 'KNR' é apenas um identificador, então deve ser removido
y = df['FALHA']

In [9]:
# Define LSTM classifier model
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Separar os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir o tamanho da sequência
sequence_length = 10  # Ajuste este valor conforme necessário

# Função para criar sequências
def create_sequences(data, seq_length):
    sequences = []
    for i in range(len(data) - seq_length + 1):
        seq = data[i:i + seq_length]
        sequences.append(seq)
    return np.array(sequences)

# Convert DataFrames to NumPy arrays
X_train_np = X_train.values
X_test_np = X_test.values

# Criar sequências para os dados de treino e teste
X_train_sequences = create_sequences(X_train_np, sequence_length)
X_test_sequences = create_sequences(X_test_np, sequence_length)

# Ajustar os rótulos para que correspondam às sequências
y_train_adjusted = y_train[sequence_length - 1:]
y_test_adjusted = y_test[sequence_length - 1:]

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_sequences, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_adjusted.values, dtype=torch.long)
X_test_tensor = torch.tensor(X_test_sequences, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_adjusted.values, dtype=torch.long)

# Define model parameters
input_size = X_train_tensor.shape[2]  # This is the number of features
hidden_size = 64
num_layers = 4
output_size = 2  # Number of classes

# Instantiate the model
model = LSTMClassifier(input_size, hidden_size, num_layers, output_size)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 50
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')


Epoch [1/50], Loss: 0.6987035274505615
Epoch [2/50], Loss: 0.6963377594947815
Epoch [3/50], Loss: 0.6945045590400696
Epoch [4/50], Loss: 0.693189799785614
Epoch [5/50], Loss: 0.6924191117286682
Epoch [6/50], Loss: 0.6922348141670227
Epoch [7/50], Loss: 0.6925681829452515
Epoch [8/50], Loss: 0.6929979920387268
Epoch [9/50], Loss: 0.6931416392326355
Epoch [10/50], Loss: 0.6930086612701416
Epoch [11/50], Loss: 0.6927550435066223
Epoch [12/50], Loss: 0.6925087571144104
Epoch [13/50], Loss: 0.6923336386680603
Epoch [14/50], Loss: 0.6922429800033569
Epoch [15/50], Loss: 0.6922227740287781
Epoch [16/50], Loss: 0.692248523235321
Epoch [17/50], Loss: 0.6922956705093384
Epoch [18/50], Loss: 0.6923444867134094
Epoch [19/50], Loss: 0.6923816204071045
Epoch [20/50], Loss: 0.6924005746841431
Epoch [21/50], Loss: 0.692400336265564
Epoch [22/50], Loss: 0.6923832893371582
Epoch [23/50], Loss: 0.692354679107666
Epoch [24/50], Loss: 0.6923202276229858
Epoch [25/50], Loss: 0.6922856569290161
Epoch [26/50]

In [12]:
# Evaluate the model on the test set
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Disable gradient calculation
    test_outputs = model(X_test_tensor)
    _, predicted = torch.max(test_outputs, 1)

# Convert to CPU for compatibility with sklearn (if using GPU)
y_test_np = y_test_tensor.cpu().numpy()
predicted_np = predicted.cpu().numpy()

# Calculate metrics
accuracy = accuracy_score(y_test_np, predicted_np)
precision = precision_score(y_test_np, predicted_np, average='weighted')
recall = recall_score(y_test_np, predicted_np, average='weighted')
f1 = f1_score(y_test_np, predicted_np, average='weighted')

# Print metrics
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

Accuracy: 0.5210
Precision: 0.2714
Recall: 0.5210
F1 Score: 0.3569


C:\Users\Inteli\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Teste LSTM 2 

In [16]:

# Normalização dos dados
scaler = MinMaxScaler(feature_range=(-1, 1))
features_scaled = scaler.fit_transform(X)

# Convertendo para tensores do PyTorch
X = torch.tensor(features_scaled, dtype=torch.float32)
y = torch.tensor(y.values, dtype=torch.float32)

# Criando sequências para alimentar o LSTM
def create_sequences(input_data, target_data, seq_length):
    sequences = []
    labels = []
    for i in range(len(input_data) - seq_length):
        seq = input_data[i:i+seq_length]
        label = target_data[i+seq_length]
        sequences.append(seq)
        labels.append(label)
    return torch.stack(sequences), torch.tensor(labels, dtype=torch.float32)

seq_length = 10  # tamanho da sequência que você quer usar
X_seq, y_seq = create_sequences(X, y, seq_length)

# Dividindo os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

# Criando DataLoader
train_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)


In [17]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_layer_size, output_size, num_layers):
        super(LSTMModel, self).__init__()
        self.hidden_layer_size = hidden_layer_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size, hidden_layer_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_layer_size, output_size)
    
    def forward(self, input_seq):
        h_0 = torch.zeros(self.num_layers, input_seq.size(0), self.hidden_layer_size).to(input_seq.device)
        c_0 = torch.zeros(self.num_layers, input_seq.size(0), self.hidden_layer_size).to(input_seq.device)
        
        lstm_out, _ = self.lstm(input_seq, (h_0, c_0))
        lstm_out = lstm_out[:, -1, :]
        predictions = self.linear(lstm_out)
        return predictions


In [18]:
# Inicializando o modelo, critério e otimizador
input_size = X_train_tensor.shape[2]
hidden_layer_size = 100
output_size = 1
num_layers = 2

model = LSTMModel(input_size, hidden_layer_size, output_size, num_layers)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Treinamento
epochs = 50
for epoch in range(epochs):
    for sequences, labels in train_loader:
        optimizer.zero_grad()
        y_pred = model(sequences)
        loss = criterion(y_pred, labels)
        loss.backward()
        optimizer.step()
    
    print(f'Epoch {epoch+1} Loss: {loss.item()}')


C:\Users\Inteli\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\Inteli\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([48])) that is different to the input size (torch.Size([48, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1 Loss: 0.24669140577316284
Epoch 2 Loss: 0.25572341680526733
Epoch 3 Loss: 0.24614101648330688
Epoch 4 Loss: 0.24992632865905762
Epoch 5 Loss: 0.24965673685073853
Epoch 6 Loss: 0.2538127303123474
Epoch 7 Loss: 0.24904002249240875
Epoch 8 Loss: 0.23685145378112793
Epoch 9 Loss: 0.2577451169490814
Epoch 10 Loss: 0.25314754247665405
Epoch 11 Loss: 0.25118911266326904
Epoch 12 Loss: 0.2500854432582855
Epoch 13 Loss: 0.25044769048690796
Epoch 14 Loss: 0.24956880509853363
Epoch 15 Loss: 0.24274642765522003
Epoch 16 Loss: 0.2482810616493225
Epoch 17 Loss: 0.24960801005363464
Epoch 18 Loss: 0.2509467899799347
Epoch 19 Loss: 0.24828556180000305
Epoch 20 Loss: 0.25002971291542053
Epoch 21 Loss: 0.2517930865287781
Epoch 22 Loss: 0.24763764441013336
Epoch 23 Loss: 0.2529253363609314
Epoch 24 Loss: 0.2500981390476227
Epoch 25 Loss: 0.25619420409202576
Epoch 26 Loss: 0.2496138960123062
Epoch 27 Loss: 0.24566470086574554
Epoch 28 Loss: 0.2491593062877655
Epoch 29 Loss: 0.2427350878715515
Epoch

In [ ]:
model.eval()
test_predictions = []

with torch.no_grad():
    for sequences, labels in test_loader:
        y_pred = model(sequences)
        test_predictions.append(y_pred)

test_predictions = torch.cat(test_predictions)


Acurácia: 0.5149
Precisão: 0.0000
Recall: 0.0000
F1-Score: 0.0000


C:\Users\Inteli\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
